# Assignment 1: This is the title of the assignment

Authors:  
Arin Rahim 

## Imports

In [16]:
# General imports
import os
import cv2 
import mediapy as media
import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import PIL
import absl.logging 
#from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers
#from tensorflow.keras.optimizers import Adam
#from official.projects.movinet.modeling import movinet
#from official.projects.movinet.modeling import movinet_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from pycocotools.coco import COCO
from IPython.display import clear_output

tf.get_logger().setLevel('ERROR')
absl.logging.set_verbosity(absl.logging.ERROR)
mpl.rcParams.update({
    'font.size': 10,
})

In [17]:
import os
print(os.getcwd())


/Users/arinrahim/Desktop/SKOLAN/ML/Assignments/Final assignment


These two defined functions are used to preprocess the frame and extract the keypoints from it. Beacuse of a unknown reason I was not able to define these function in the utility file , so they were copied here. 

In [18]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize_with_pad(image, 192, 192)
    image = tf.cast(image, tf.int32) 
    image = tf.expand_dims(image, axis=0)
    return image


def extract_keypoints_from_movenet(image, movenet):
    input_image = preprocess_image(image)
    outputs = movenet(input_image)
    keypoints = outputs['output_0']
    return keypoints.numpy()


To go through the data correctly, os-functions were used to find the MP4 videos of each activity and loop through them. OpenCV was utillized to extract frames from the video and  Movenet was then used for pose estimation on these videos, specifically the lighting variant. This cell took a while to run and to avoid running it again , I have provide you with the data in the folder processes/pre_processes. 

In [81]:
# Define the keypoint names
keypoint_names = [
    "nose", "left_eye",
    "right_eye", "left_ear",
    "right_ear", "left_shoulder",
    "right_shoulder", "left_elbow", 
    "right_elbow", "left_wrist",
    "right_wrist", "left_hip",
    "right_hip", "left_knee", 
    "right_knee", "left_ankle",
    "right_ankle"
]

# Define the activites number
activity_to_number = {
    'pushup' : 1, 
    'curl' : 2,
    'fly' : 3,
    'squat' : 4,
    'birddog' : 5, 
    'superman' : 6,
    'bicyclecrunch' : 7,
    'legraise' : 8, 
    'armraise' : 9,
    'overheadpress' : 10
    }

# Load model 
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']
  
 # List created to store the keypoints data for each frame of video
data = []
video_number = 0

# Base directory where the folders are located
base_dir = 'raw_data'


# Loop through each main folder in the base directory
for main_folder in os.listdir(base_dir):
    # Construct the path to the first-level exercise folder
    first_level_path = os.path.join(base_dir, main_folder)
    
    # Check if it's a directory to avoid processing any files directly under base_dir
    if os.path.isdir(first_level_path):
        
        # Get the list of subdirectories (should be just one per your structure)
        subdirectories = [d for d in os.listdir(first_level_path) if os.path.isdir(os.path.join(first_level_path, d))]
        
        # Check if there's at least one subdirectory and proceed
        if subdirectories:
            # Path to the actual folder containing MP4 files (second-level folder)
            video_folder_path = os.path.join(first_level_path, subdirectories[0])
            
            # Loop through each MP4 file in the video folder
            for video_file in os.listdir(video_folder_path):
                video_path = os.path.join(video_folder_path, video_file)
                
                # Check if the file is an MP4 video
                if os.path.isfile(video_path) and video_file.endswith('.mp4'):
                    print(f"Processing {video_file} in {main_folder}...")
                    video_number = video_number + 1
                    print("Video number: ", video_number)
                    cap = cv2.VideoCapture(video_path)
                    frame_number = 0
                    
                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret:
                            break
                        
                        frame_number += 1
                        #print("Processing: frame: ", frame_number)
                        keypoints = extract_keypoints_from_movenet(frame, movenet)
                        
                        #keypoints_flat = keypoints.flatten().tolist()
                        activity_name = subdirectories[0]
                        activity_label = activity_to_number[activity_name]
                        record = {'Video' : f'{video_file}', 'activity_label': activity_label, 'frame_number': frame_number}

                        # Append data including frame number, exercise label, and keypoints
                        for i, kp in enumerate(keypoints[0, 0, :, :]): 
                            body_part = keypoint_names[i]
                            x, y, confidence = kp[1], kp[0], kp[2] 
                            
                            record[f'k_pts_{body_part}_x'] = x
                            record[f'k_pts_{body_part}_y'] = y
                            record[f'k_pts_{body_part}_confidence'] = confidence
                        #print(record)
                        data.append(record)
                        #print(data)
                        
                            
                    cap.release()

Processing 000038.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  1


2024-03-06 17:28:24.317096: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Processing 000010.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  2
Processing 000004.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  3
Processing 000005.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  4
Processing 000011.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  5
Processing 000039.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  6
Processing 000007.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  7
Processing 000013.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  8
Processing 000012.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  9
Processing 000006.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  10
Processing 000002.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  11
Processing 000016.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  12
Processing 000017.mp4 in infinityai_fitness_basic_curl_v1.0...
Video number:  13
Processing 000003.mp4 in infinityai_

## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [19]:
# Stores the keypoint data in a dataframe and saves it to a CSV file. The file is saved in processes/preprocess_data 
#df = pd.DataFrame(data)
#df_activity = df.copy()

# Read the file directly to avoid re-running previous step
df = pd.read_csv('processes/preprocess_data/PoseEstimationData.csv')
df_activity = df.copy()


In [20]:
# Prints the first 5 rows of the dataframe
df_activity.head()

,Video,activity_label,frame_number,k_pts_nose_x,k_pts_nose_y,k_pts_nose_confidence,k_pts_left_eye_x,k_pts_left_eye_y,k_pts_left_eye_confidence,k_pts_right_eye_x,...,k_pts_left_knee_confidence,k_pts_right_knee_x,k_pts_right_knee_y,k_pts_right_knee_confidence,k_pts_left_ankle_x,k_pts_left_ankle_y,k_pts_left_ankle_confidence,k_pts_right_ankle_x,k_pts_right_ankle_y,k_pts_right_ankle_confidence
0,000038.mp4,2,1,0.375629,0.329855,0.575082,0.379877,0.318692,0.671976,0.358221,...,0.725594,0.282331,0.859699,0.834232,0.346216,0.977464,0.585235,0.245951,0.980421,0.492093
1,000038.mp4,2,2,0.374021,0.329474,0.556815,0.378901,0.318824,0.653531,0.356590,...,0.736575,0.280325,0.862176,0.834284,0.346497,0.976944,0.531707,0.245042,0.980027,0.474869
2,000038.mp4,2,3,0.378061,0.328498,0.627583,0.379186,0.318344,0.656229,0.361404,...,0.738356,0.281732,0.861353,0.847008,0.346142,0.977287,0.577636,0.246038,0.979141,0.484306
3,000038.mp4,2,4,0.378496,0.326863,0.658308,0.379104,0.316471,0.618782,0.361762,...,0.712585,0.282561,0.862615,0.841878,0.346289,0.977373,0.515678,0.245476,0.980479,0.477897
4,000038.mp4,2,5,0.377487,0.327366,0.627432,0.378026,0.317184,0.572762,0.362475,...,0.721487,0.278926,0.864761,0.838640,0.346617,0.977318,0.554312,0.245807,0.980555,0.483587


## Task 1: Train model with pose estimation data

The model that was choosen to train and use for predicting the different activities was the SVM-model. The choice of model was based on errors that hindered me from using a neural network such as LSTM, and since time was short I wanted something simple and robust, therefore SVM seemed like an appropriate choice. 

In [124]:

# Define the split data 
X = df_activity.drop(['activity_label', 'Video', 'frame_number'], axis=1)
y = df_activity['activity_label']

# Split the dataset into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [125]:
# Create the svm model, and use the pipeline to scale the data
svm_model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10))

# Train the model using the training set
svm_model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC(C=10))])

In [127]:
# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Detailed performance report
print(classification_report(y_test, y_pred))

Accuracy: 0.94
              precision    recall  f1-score   support

           1       0.92      0.90      0.91     10221
           2       0.98      0.99      0.98     10534
           3       0.98      0.98      0.98      6565
           4       0.98      0.97      0.97     11793
           5       0.94      0.96      0.95      8710
           6       0.89      0.94      0.91     12815
           7       0.88      0.88      0.88      6971
           8       0.90      0.85      0.87     10263
           9       0.97      0.98      0.98     13402
          10       0.98      0.99      0.98     11301

    accuracy                           0.94    102575
   macro avg       0.94      0.94      0.94    102575
weighted avg       0.94      0.94      0.94    102575



## Task 2: Feature engineering 

For task 1, a total of 51 features was used by the model including the keypoints of each joints left and right side, and the confidence score. For task 2, feature engineering was applied to extract more information from these raw data. Distance between the keypoints, distance ratio, and some joint angles were caluclated by using three functions. As I mentioned previously, the utility file did not work so the functions for calculating distance and angle are in the main script. 

Same reason here, I got errors when trying to use a LSTM-model so SVM was utilized again. 

In [91]:
# Function for calculating the distance between two keypoints
def distance_keypoints(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

# Function for calculating the angle between three keypoints
def angle_keypoints(ax, ay, bx, by, cx, cy):
    ba = [ax - bx, ay - by]
    bc = [cx - bx, cy - by]
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)
    
# Create a copy of the original data frame
df_activity_features = df.copy()

# An empty list to store the calculated features
features = []

for _, row in df_activity_features.iterrows():
    
    # Calculate distance between different joints for left and right side 
    
    distance_wrist_shoulder_left = distance_keypoints(row['k_pts_left_wrist_x'], row['k_pts_left_wrist_y'],
                                                 row['k_pts_left_shoulder_x'], row['k_pts_left_shoulder_y'])
    
    distance_wrist_shoulder_right = distance_keypoints(row['k_pts_right_wrist_x'], row['k_pts_right_wrist_y'],
                                                 row['k_pts_right_shoulder_x'], row['k_pts_right_shoulder_y'])
    
    distance_hip_ankle_left = distance_keypoints(row['k_pts_left_hip_x'], row['k_pts_left_hip_y'],
                                            row['k_pts_left_ankle_x'], row['k_pts_left_ankle_y'])
    
    distance_hip_ankle_right = distance_keypoints(row['k_pts_right_hip_x'], row['k_pts_right_hip_y'],
                                            row['k_pts_right_ankle_x'], row['k_pts_right_ankle_y'])
    
    distance_nose_ankle_left = distance_keypoints(row['k_pts_nose_x'], row['k_pts_nose_y'],
                                                row['k_pts_left_ankle_x'], row['k_pts_left_ankle_y'])
    
    distance_nose_ankle_right = distance_keypoints(row['k_pts_nose_x'], row['k_pts_nose_y'],
                                                   row['k_pts_right_ankle_x'], row['k_pts_right_ankle_y'])
    
    distance_nose_knee_left = distance_keypoints(row['k_pts_nose_x'], row['k_pts_nose_y'], 
                                                 row['k_pts_left_knee_x'], row['k_pts_left_knee_y'])
    
    distance_nose_knee_right = distance_keypoints(row['k_pts_nose_x'], row['k_pts_nose_y'], 
                                                  row['k_pts_right_knee_x'], row['k_pts_right_knee_y'])
    
    distance_nose_wrist_left = distance_keypoints(row['k_pts_nose_x'], row['k_pts_nose_y'], 
                                                   row['k_pts_left_wrist_x'], row['k_pts_left_wrist_y'])
    
    distance_nose_wrist_right = distance_keypoints(row['k_pts_nose_x'], row['k_pts_nose_y'], 
                                                   row['k_pts_right_wrist_x'], row['k_pts_right_wrist_y'])
    
    # Calculate angles for different joints for left and right side 
    angle_elbow_left = angle_keypoints(row['k_pts_left_shoulder_x'], row['k_pts_left_shoulder_y'],
                                  row['k_pts_left_elbow_x'], row['k_pts_left_elbow_y'],
                                  row['k_pts_left_wrist_x'], row['k_pts_left_wrist_y'])
    
    angle_elbow_right = angle_keypoints(row['k_pts_right_shoulder_x'], row['k_pts_right_shoulder_y'],
                                  row['k_pts_right_elbow_x'], row['k_pts_right_elbow_y'],
                                  row['k_pts_right_wrist_x'], row['k_pts_right_wrist_y'])
    
    
    angle_knee_left = angle_keypoints(row['k_pts_left_hip_x'], row['k_pts_left_hip_y'],
                                      row['k_pts_left_knee_x'], row['k_pts_left_knee_y'],
                                      row['k_pts_left_ankle_x'], row['k_pts_left_ankle_y'])
    
    angle_knee_right = angle_keypoints(row['k_pts_right_hip_x'], row['k_pts_right_hip_y'],
                                       row['k_pts_right_knee_x'], row['k_pts_right_knee_y'], 
                                       row['k_pts_right_ankle_x'], row['k_pts_right_ankle_y'])
    
    angle_hip_right = angle_keypoints(row['k_pts_right_shoulder_x'], row['k_pts_right_shoulder_y'], 
                                      row['k_pts_right_hip_x'], row['k_pts_right_hip_y'], 
                                      row['k_pts_right_knee_x'], row['k_pts_right_knee_y'])
    
    angle_hip_left = angle_keypoints(row['k_pts_left_shoulder_x'], row['k_pts_left_shoulder_y'], 
                                     row['k_pts_left_hip_x'], row['k_pts_left_hip_y'], 
                                     row['k_pts_left_knee_x'], row['k_pts_left_knee_y'])
    
    # Keeping track of the head during the activites 
    head_position_x = row['k_pts_nose_x']
    head_position_y = row['k_pts_nose_y']
    eye_left_x = row['k_pts_left_eye_x']
    eye_left_y = row['k_pts_left_eye_y']
    eye_right_x = row['k_pts_right_eye_x']
    eye_right_y = row['k_pts_right_eye_y']
    
    # Extra features 
    feature1 = row['k_pts_left_shoulder_y'] ** 2 - row['k_pts_right_ankle_x']
    feature2 = np.arctan2(row['k_pts_left_hip_x'] - row['k_pts_right_elbow_y'], row['k_pts_right_shoulder_y'] - row['k_pts_left_knee_x'])
    feature3 = row['k_pts_right_knee_x'] * row['k_pts_left_ankle_y']
    feature4 = np.exp(row['k_pts_right_elbow_x'] - row['k_pts_left_knee_y'])
    feature5 = np.log(abs(row['k_pts_left_wrist_x'] - row['k_pts_right_ankle_y']) + 1)
      
    # Append the calculated features for each row into the list
    features.append({
            'Video': row['Video'],
            'Frame': row['frame_number'],
            'activity_label': row['activity_label'],
            'distance_wrist_shoulder_left' : distance_wrist_shoulder_left,
            'distance_wrist_shoulder_right' : distance_wrist_shoulder_right,
            'distance_hip_ankle_left' : distance_hip_ankle_left,
            'distance_hip_ankle_right' : distance_hip_ankle_right,
            'distance_nose_ankle_left' : distance_nose_ankle_left,
            'distance_nose_ankle_right' : distance_nose_ankle_right,
            'distance_nose_knee_left' : distance_nose_knee_left,
            'distance_nose_knee_right' : distance_nose_knee_right,
            'distance_nose_wrist' : distance_nose_wrist_left,
            'distance_nose_wrist_right' : distance_nose_wrist_right,
            'head_position_x' : head_position_x,
            'head_position_y' : head_position_y,
            'eye_left_x': eye_left_x,
            'eye_left_y': eye_left_y,
            'eye_right_x': eye_right_x,
            'eye_right_y': eye_right_y,
            'feature1': feature1,
            'feature2': feature2,
            'feature3': feature3,
            'feature4': feature4,
            'feature5': feature5,
            'angle_elbow_left': angle_elbow_left,
            'angle_elbow_right' : angle_elbow_right,
            'angle_knee_left' : angle_knee_left,
            'angle_knee_right' : angle_knee_right,
            'angle_hip_right' : angle_hip_right,
            'angle_hip_left' : angle_hip_left
        })


In [92]:
# Create a dataframe from the list of calculated features, and stores it in the processes/preprocess_data folder
df_calculated_features = pd.DataFrame(features)
df_features = df_calculated_features.copy()
df_features.to_csv('processes/preprocess_data/CalculatedFeaturesData.csv', index=False)


In [93]:
X_task2 = df_features.drop(['activity_label', 'Video', 'Frame'], axis=1) # features 
y_task2 = df_features['activity_label'] # labels 

# Split the dataset into training and testing sets (70% for training, 30% for testing)
X_train_task2, X_test_task2, y_train_task2, y_test_task2 = train_test_split(X_task2, y_task2, test_size=0.3, random_state=42)



In [94]:
svm_model_task2 = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10))
svm_model_task2.fit(X_train_task2, y_train_task2)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC(C=10))])

In [95]:
# Make predictions on the test set
y_pred_task2 = svm_model_task2.predict(X_test_task2)

# Calculate the accuracy
accuracy_task2 = accuracy_score(y_test_task2, y_pred_task2)
print(f"Accuracy: {accuracy_task2:.2f}")

# Detailed performance report
print(classification_report(y_test_task2, y_pred_task2))

Accuracy: 0.90
              precision    recall  f1-score   support

           1       0.87      0.83      0.85     10221
           2       0.95      0.96      0.96     10534
           3       0.95      0.95      0.95      6565
           4       0.93      0.92      0.93     11793
           5       0.89      0.93      0.91      8710
           6       0.84      0.89      0.86     12815
           7       0.80      0.80      0.80      6971
           8       0.85      0.76      0.80     10263
           9       0.93      0.93      0.93     13402
          10       0.95      0.96      0.95     11301

    accuracy                           0.90    102575
   macro avg       0.90      0.89      0.89    102575
weighted avg       0.90      0.90      0.90    102575



## Task 3: MoViNet model

MoViNet model is a tensorflow model used to predict the activity of human movements. Data from 600 types of activites are collected, including some labels which we have no corresponding video data for (e.g., belly dancing, streching arm, zumba). Ive divided task 3 into two seperate ways to use the MoViNet model for prediction. For both of the tasks im only using one video for armraise (000000.mp4)

All the code can be found here for the first task: https://www.tensorflow.org/hub/tutorials/movinet 

All the code can be found here for the second task: https://github.com/PranayLendave/video_classification/blob/main/trained_movinets_notebook.ipynb 

### Task 3.1 

In [53]:
model_handle_task3 = "https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/3"
model_task3 = hub.load(model_handle_task3)
sig = model_task3.signatures['serving_default']


In [55]:
print(sig.pretty_printed_signature())


Input Parameters:
  image (KEYWORD_ONLY): TensorSpec(shape=(None, None, None, None, 3), dtype=tf.float32, name='image')
Output Type:
  Dict[['classifier_head', TensorSpec(shape=(None, 600), dtype=tf.float32, name='classifier_head')]]
Captures:
  16249786384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119903408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119903232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119902880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119903056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  16249786032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119902704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119902176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119902352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119902528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  16249786208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  23119902000: TensorSpec(shap

In [25]:
my_labels = ['pushup', 'curl', 'fly', 'squat', 'birddog', 'superman', 'bicyclecrunch', 'legraise', 'armraise', 'overheadpress']

video_path = 'raw_data/infinityai_fitness_basic_armraise_v1.0/armraise/000000.mp4'
frames = []
cap_task3 = cv2.VideoCapture(video_path)

while cap_task3.isOpened():
    
    ret, frame = cap_task3.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = tf.image.resize(frame, [224, 224])
    frame = tf.cast(frame, tf.float32) / 255.
    #frame = tf.expand_dims(frame, axis=0)
    frames.append(frame)
    
cap_task3.release()

frames_stack = tf.stack(frames)
frames_batch = frames_stack[tf.newaxis, ...]

logits = sig(image = frames_batch)
logits = logits['classifier_head'][0]
print(logits)
#sig(image = frames_stack[tf.newaxis, :1])
#frames_batch = tf.stack(frames)
#print(frames)
#print(sig(image = frames_batch[tf.newaxis, ...]))
#logits = sig(image = frames_batch[tf.newaxis, ...])
#logits = logits['classifier_head'][0]


#predictions = model_task3(frames_batch)
#predicted_label = my_labels[np.argmax(predictions)]
#print(predicted_label)
#clear_output(wait=True)

I0000 00:00:1710717359.449051 17024035 service.cc:145] XLA service 0x3066ca830 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710717359.449648 17024035 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1710717359.588260 17024035 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [26]:
print(sig(image = frames_stack[tf.newaxis, :1]))

{'classifier_head': <tf.Tensor: shape=(1, 600), dtype=float32, numpy=
array([[-1.9094306 , -2.5886474 ,  1.6810025 , -2.2969475 , -2.3116817 ,
         3.045559  ,  2.2881444 ,  2.0337608 ,  1.4685065 , -2.4415512 ,
         2.24522   ,  0.5677395 ,  0.07643534,  1.3814644 , -0.5900388 ,
         2.1788502 ,  0.2171168 , -1.0539699 , -0.7459621 ,  1.6103688 ,
         2.1918404 , -2.0405083 ,  0.32173687, -0.08703735,  0.4316071 ,
        -0.54029   , -0.9333844 , -3.4491863 ,  1.1245267 , -1.9266937 ,
         0.23032688, -0.03504841, -1.6777468 ,  0.6376343 , -0.3145659 ,
        -3.604264  ,  1.4723319 , -0.25907606,  0.8605443 , -2.473881  ,
        -1.843982  ,  3.8969946 , -1.2075691 , -0.8539759 ,  1.1625438 ,
         2.5533142 ,  0.7679703 ,  3.559575  , -2.5840201 ,  0.16140175,
         2.5473695 ,  1.3964581 ,  0.0905668 ,  1.7405701 , -0.30984348,
         0.3735016 ,  0.09618336,  1.2752264 , -0.611975  , -0.1277915 ,
        -1.6958466 , -0.37772623, -1.214918  , -0.0573

In [34]:
predictions = model_task3(frames_batch)
predicted_label = my_labels[np.argmax(predictions)]
print(predicted_label)
clear_output(wait=True)

ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * <tf.Tensor 'inputs:0' shape=(1, 578, 224, 224, 3) dtype=float32>
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * {'image': TensorSpec(shape=(None, None, None, None, 3), dtype=tf.float32, name='image')}
    * False
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * {'image': TensorSpec(shape=(None, None, None, None, 3), dtype=tf.float32, name='inputs/image')}
    * False
    * None
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * {'image': TensorSpec(shape=(None, None, None, None, 3), dtype=tf.float32, name='inputs/image')}
    * True
    * None
  Keyword arguments: {}

Option 4:
  Positional arguments (3 total):
    * {'image': TensorSpec(shape=(None, None, None, None, 3), dt

### Task 3.2

In [28]:
#@title Run this cell to set up some helper code.

# Download Kinetics 600 label map
!wget https://raw.githubusercontent.com/tensorflow/models/f8af2291cced43fc9f1d9b41ddbf772ae7b0d7d2/official/projects/movinet/files/kinetics_600_labels.txt -O labels.txt -q

with tf.io.gfile.GFile('labels.txt') as f:
  lines = f.readlines()
  KINETICS_600_LABELS_LIST = [line.strip() for line in lines]
  KINETICS_600_LABELS = tf.constant(KINETICS_600_LABELS_LIST)

def get_top_k(probs, k=5, label_map=KINETICS_600_LABELS):
  """Outputs the top k model labels and probabilities on the given video."""
  top_predictions = tf.argsort(probs, axis=-1, direction='DESCENDING')[:k]
  top_labels = tf.gather(label_map, top_predictions, axis=-1)
  top_labels = [label.decode('utf8') for label in top_labels.numpy()]
  top_probs = tf.gather(probs, top_predictions, axis=-1).numpy()
  return tuple(zip(top_labels, top_probs))

def predict_top_k(model, video, k=5, label_map=KINETICS_600_LABELS):
  """Outputs the top k model labels and probabilities on the given video."""
  outputs = model.predict(video[tf.newaxis])[0]
  probs = tf.nn.softmax(outputs)
  return get_top_k(probs, k=k, label_map=label_map)

def load_movinet_from_hub(model_id, model_mode, hub_version=3):
  """Loads a MoViNet model from TF Hub."""
  hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/{model_mode}/kinetics-600/classification/{hub_version}'

  encoder = hub.KerasLayer(hub_url, trainable=True)

  inputs = tf.keras.layers.Input(
      shape=[None, None, None, 3],
      dtype=tf.float32)

  if model_mode == 'base':
    inputs = dict(image=inputs)
  else:
    # Define the state inputs, which is a dict that maps state names to tensors.
    init_states_fn = encoder.resolved_object.signatures['init_states']
    state_shapes = {
        name: ([s if s > 0 else None for s in state.shape], state.dtype)
        for name, state in init_states_fn(tf.constant([0, 0, 0, 0, 3])).items()
    }
    states_input = {
        name: tf.keras.Input(shape[1:], dtype=dtype, name=name)
        for name, (shape, dtype) in state_shapes.items()
    }

    # The inputs to the model are the states and the video
    inputs = {**states_input, 'image': inputs}

  # Output shape: [batch_size, 600]
  outputs = encoder(inputs)

  model = tf.keras.Model(inputs, outputs)
  model.build([1, 1, 1, 1, 3])

  return model

# Download example gif
!wget https://github.com/tensorflow/models/raw/f8af2291cced43fc9f1d9b41ddbf772ae7b0d7d2/official/projects/movinet/files/jumpingjack.gif -O jumpingjack.gif -q

def load_gif(file_path, image_size=(224, 224)):
  """Loads a gif file into a TF tensor."""
  with tf.io.gfile.GFile(file_path, 'rb') as f:
    video = tf.io.decode_gif(f.read())
  video = tf.image.resize(video, image_size)
  video = tf.cast(video, tf.float32) / 255.
  return video

def get_top_k_streaming_labels(probs, k=5, label_map=KINETICS_600_LABELS_LIST):
  """Returns the top-k labels over an entire video sequence.

  Args:
    probs: probability tensor of shape (num_frames, num_classes) that represents
      the probability of each class on each frame.
    k: the number of top predictions to select.
    label_map: a list of labels to map logit indices to label strings.

  Returns:
    a tuple of the top-k probabilities, labels, and logit indices
  """
  top_categories_last = tf.argsort(probs, -1, 'DESCENDING')[-1, :1]
  categories = tf.argsort(probs, -1, 'DESCENDING')[:, :k]
  categories = tf.reshape(categories, [-1])

  counts = sorted([
      (i.numpy(), tf.reduce_sum(tf.cast(categories == i, tf.int32)).numpy())
      for i in tf.unique(categories)[0]
  ], key=lambda x: x[1], reverse=True)

  top_probs_idx = tf.constant([i for i, _ in counts[:k]])
  top_probs_idx = tf.concat([top_categories_last, top_probs_idx], 0)
  top_probs_idx = tf.unique(top_probs_idx)[0][:k+1]

  top_probs = tf.gather(probs, top_probs_idx, axis=-1)
  top_probs = tf.transpose(top_probs, perm=(1, 0))
  top_labels = tf.gather(label_map, top_probs_idx, axis=0)
  top_labels = [label.decode('utf8') for label in top_labels.numpy()]

  return top_probs, top_labels, top_probs_idx

def plot_streaming_top_preds_at_step(
    top_probs,
    top_labels,
    step=None,
    image=None,
    legend_loc='lower left',
    duration_seconds=10,
    figure_height=500,
    playhead_scale=0.8,
    grid_alpha=0.3):
  """Generates a plot of the top video model predictions at a given time step.

  Args:
    top_probs: a tensor of shape (k, num_frames) representing the top-k
      probabilities over all frames.
    top_labels: a list of length k that represents the top-k label strings.
    step: the current time step in the range [0, num_frames].
    image: the image frame to display at the current time step.
    legend_loc: the placement location of the legend.
    duration_seconds: the total duration of the video.
    figure_height: the output figure height.
    playhead_scale: scale value for the playhead.
    grid_alpha: alpha value for the gridlines.

  Returns:
    A tuple of the output numpy image, figure, and axes.
  """
  num_labels, num_frames = top_probs.shape
  if step is None:
    step = num_frames

  fig = plt.figure(figsize=(6.5, 7), dpi=300)
  gs = mpl.gridspec.GridSpec(8, 1)
  ax2 = plt.subplot(gs[:-3, :])
  ax = plt.subplot(gs[-3:, :])

  if image is not None:
    ax2.imshow(image, interpolation='nearest')
    ax2.axis('off')

  preview_line_x = tf.linspace(0., duration_seconds, num_frames)
  preview_line_y = top_probs

  line_x = preview_line_x[:step+1]
  line_y = preview_line_y[:, :step+1]

  for i in range(num_labels):
    ax.plot(preview_line_x, preview_line_y[i], label=None, linewidth='1.5',
            linestyle=':', color='gray')
    ax.plot(line_x, line_y[i], label=top_labels[i], linewidth='2.0')


  ax.grid(which='major', linestyle=':', linewidth='1.0', alpha=grid_alpha)
  ax.grid(which='minor', linestyle=':', linewidth='0.5', alpha=grid_alpha)

  min_height = tf.reduce_min(top_probs) * playhead_scale
  max_height = tf.reduce_max(top_probs)
  ax.vlines(preview_line_x[step], min_height, max_height, colors='red')
  ax.scatter(preview_line_x[step], max_height, color='red')

  ax.legend(loc=legend_loc)

  plt.xlim(0, duration_seconds)
  plt.ylabel('Probability')
  plt.xlabel('Time (s)')
  plt.yscale('log')

  fig.tight_layout()
  fig.canvas.draw()

  data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
  plt.close()

  figure_width = int(figure_height * data.shape[1] / data.shape[0])
  image = PIL.Image.fromarray(data).resize([figure_width, figure_height])
  image = np.array(image)

  return image, (fig, ax, ax2)

def plot_streaming_top_preds(
    probs,
    video,
    top_k=5,
    video_fps=25.,
    figure_height=500,
    use_progbar=True):
  """Generates a video plot of the top video model predictions.

  Args:
    probs: probability tensor of shape (num_frames, num_classes) that represents
      the probability of each class on each frame.
    video: the video to display in the plot.
    top_k: the number of top predictions to select.
    video_fps: the input video fps.
    figure_fps: the output video fps.
    figure_height: the height of the output video.
    use_progbar: display a progress bar.

  Returns:
    A numpy array representing the output video.
  """
  video_fps = 8.
  figure_height = 500
  steps = video.shape[0]
  duration = steps / video_fps

  top_probs, top_labels, _ = get_top_k_streaming_labels(probs, k=top_k)

  images = []
  step_generator = tqdm.trange(steps) if use_progbar else range(steps)
  for i in step_generator:
    image, _ = plot_streaming_top_preds_at_step(
        top_probs=top_probs,
        top_labels=top_labels,
        step=i,
        image=video[i],
        duration_seconds=duration,
        figure_height=figure_height,
    )
    images.append(image)

  return np.array(images)

In [35]:
model = load_movinet_from_hub('a2', 'base', hub_version=3)

ValueError: Exception encountered when calling layer 'keras_layer_2' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_2' (type KerasLayer):
  • inputs={'image': '<KerasTensor shape=(None, None, None, None, 3), dtype=float32, sparse=None, name=keras_tensor_2>'}
  • training=None

## Results and Discussion

## Task 1

The extraction of keypoints from the videos for different activities went as expected. The MoveNet model was used to detect human poses and extract keypoints along with their respective confidence values for each frame of every video. The data extraction took time, but having an organized dataset makes training and fitting a model easier. As mentioned earlier, errors prevented me from using an LSTM, and an SVM model was used instead. The SVM model performed well and reported an accuracy of 94%. I'm satisfied with the accuracy, but since the dataset is considered large and avoiding overfitting with an LSTM can be done by using early stopping, I would have preferred using a neural network over the SVM model to have a more robust model with the possibility to manage overfitting scenarios. Down below are the classification report for the SVM model: 

              precision    recall  f1-score   support

           1       0.92      0.90      0.91     10221
           2       0.98      0.99      0.98     10534
           3       0.98      0.98      0.98      6565
           4       0.98      0.97      0.97     11793
           5       0.94      0.96      0.95      8710
           6       0.89      0.94      0.91     12815
           7       0.88      0.88      0.88      6971
           8       0.90      0.85      0.87     10263
           9       0.97      0.98      0.98     13402
          10       0.98      0.99      0.98     11301

    accuracy                           0.94    102575
   
   macro avg       0.94      0.94      0.94    102575

weighted avg       0.94      0.94      0.94    102575

## Task 2

Task 1 gave us a total of 51 features to use for our machine learning model. However, using  all these features at once may increases the risk of having a feature that is too dominangt, and lead to overfitting the model. Feature engineering is used to decrease the number of features that is used for our model. By using less features we decrease both the computional load and the risk of overfitting. 

Since we are trying to predict activity, I used the distance between the different joints and the angle between different joints. Calculating the distance and angle is a way to be able to tell the labels apart. For example in armraise, the angle of the elbow and the distance between the wrist and shoulder decreases, while the angle of the knee and the distance between the hip and ankle is consistent. For a different activity such as squats, the angle of the knee and the distanece between the nose and ankle both decreases. Lastly, I added some more feature that keeps track of the head during the different activites, and some other extra helpful features. 

I had the same problem as in task 1, neural network did not work so SVM model was used again. The data was scaled using StandarScaler, the kernal is used was Radial Basis Function (rbf) with a regularization parameter of 10. A total of 27 features were engineered, which is 24 fewer than in task 1. The model reported an accuracy of 90 %. Although I got a 4 % decrease in model prediction accuracy compared to task 1, considering the reduced number of features, I wouldnt state that the overall performance was significantly affected. Down below are the classification report for the SVM model: 


Minimal set of features: 27. 


Accuracy: 0.90
              
              precision    recall  f1-score   support

           1       0.87      0.83      0.85     10221
           2       0.95      0.96      0.96     10534
           3       0.95      0.95      0.95      6565
           4       0.93      0.92      0.93     11793
           5       0.89      0.93      0.91      8710
           6       0.84      0.89      0.86     12815
           7       0.80      0.80      0.80      6971
           8       0.85      0.76      0.80     10263
           9       0.93      0.93      0.93     13402
          10       0.95      0.96      0.95     11301

    accuracy                           0.90    102575
   
   
   macro avg       0.90      0.89      0.89    102575


weighted avg       0.90      0.90      0.90    102575


## Task 3

The MoViNet model is trained on a big dataset called Kinetics 600, which includes 600 different types of activities. Some of these activities involve moving parts of the body like arms and legs, bending arms, or moving wrists. Even though we didn't specifically teach the model to recognize our exact activities, it can still guess with some accuracy. For example, if our video shows someone raising their arm, the model might guess the person is doing an activity that involves arm raising. The guess might not be exactly right, but it shows the model knows which body parts are moving.

I tried to use the MoViNet model for our project, but I ran into some problems. I looked online for help on sites like Stack Overflow and GitHub but couldn't fix the issues. After spending a lot of time trying to debug without any luck, I decided to use Google Colab instead, because TensorFlow has a special link for it. You can see the results of that below:

tai chi             : 0.331

belly dancing       : 0.298

stretching arm      : 0.183

yoga                : 0.051

doing aerobics      : 0.014

The outcome wasn't quite what I expected because there's an activity called "front raises" in the Kinetics 600 dataset. I thought the model would guess something close to "front raises" for videos where someone is lifting their arms. However, it guessed activities like "tai chi" or "belly dancing" instead, which also have a lot of arm movements.


Link to the google colab: https://colab.research.google.com/drive/1MU3gp33guZEFeVzRvYBXWZPi6Dzrr66n?usp=sharing 